In [75]:
from ultralytics import YOLO
import numpy as np
import os
import cv2
from PIL import Image
import pytesseract

In [76]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


In [77]:
#trained yolo model
lp_detect_model = YOLO('yolo_model/best.pt')

In [78]:
#crop the license plate and save as image
def crop_license_plates(img, yolo_model, save_img=True):
    img_name = img.split("\\")[-1]
    
    print(f"Processing {img_name}")
    detected_lps = yolo_model.predict(img)[0]
    license_plates = []
    
    for lp in detected_lps.boxes.data.tolist():
        
        x1, y1, x2, y2, conf, _ = lp
       
        img_array = cv2.imread(img)
        cropped_lp =img_array[int(y1):int(y2), int(x1):int(x2), :]
        license_plates.append(cropped_lp)
        if save_img:
            cv2.imwrite(f"cropped_license_plates/cropped_{img_name}", cropped_lp)
        
        
    return license_plates

In [79]:
def adjust_dpi(image, target_dpi):
    # Convert the NumPy array to Pillow image
    pil_image = Image.fromarray(image)

    # Calculate the scaling factor to achieve the target DPI
    current_dpi = pil_image.info.get('dpi', (72, 72))  # Default DPI if not specified
    scaling_factor = target_dpi / current_dpi[0]  # Assuming X and Y DPI are the same

    # Calculate the new size based on the scaling factor
    new_size = (int(pil_image.width * scaling_factor), int(pil_image.height * scaling_factor))

    # Resize the image using Pillow
    resized_image = pil_image.resize(new_size, resample=Image.LANCZOS)

    # Set the DPI information
    resized_image.info['dpi'] = (target_dpi, target_dpi)

    # Convert the resized image back to a NumPy array
    resized_array = np.array(resized_image)

    return resized_array

In [122]:
def ocr(image):
    custom_config = r'--psm 6 --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
    text = pytesseract.image_to_string(image, lang='deu', config=custom_config)
    if text:
        return text.strip("\n")
    return "not detected"

In [123]:
#read characters on licenese plate using OCR  
def read_license_plates(license_plates,name, save_img=True):
    kernel = np.ones((3,3),np.uint8)
    for lp in license_plates:

        #pre processing of the image:
        
        #1.    grayscale the image
       
        lp_gray = cv2.cvtColor(lp, cv2.COLOR_BGR2GRAY)

        #2.     erode the image
        lp_gray = cv2.erode(lp_gray, kernel, iterations=1)

        #3a.     remove noise with median filter
        lp_median = cv2.medianBlur(lp_gray,5)
        
        #3b.     remove noise with bilateral filter
        lp_bilateral = cv2.bilateralFilter(lp_gray, 5, 75, 75)

        #4a.    canny edge detection 
        lp_canny = cv2.Canny(lp_median, 100, 200)

        #4b.     binary thresholding 
        _, lp_threshold = cv2.threshold(lp_median, 6, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        _, lp_threshold2 = cv2.threshold(lp_bilateral, 6, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        if save_img:
            cv2.imwrite(f'thresholded_license_plates/thresh_median_{name}', lp_threshold)
            cv2.imwrite(f'thresholded_license_plates/thresh_bil_{name}', lp_threshold2)
            cv2.imwrite(f'thresholded_license_plates/canny_{name}', lp_canny)

        return lp_threshold, lp_threshold2, lp_canny

In [124]:
test_lps = crop_license_plates('cam_frames/image_2.jpg', lp_detect_model)

median, bilateral, canny = read_license_plates(test_lps,"test.jpg")


Processing cam_frames/image_2.jpg


image 1/1 e:\Users\belau\Documents\BV-ML-CV-Praktikum\cam_frames\image_2.jpg: 512x640 1 license_plate, 168.2ms
Speed: 2.0ms preprocess, 168.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


In [129]:
path_to_cam_frames = 'cam_frames'
res = []
for f in os.listdir(path_to_cam_frames):
    if f.endswith('.jpg') or f.endswith('.png'):
        img = os.path.join(path_to_cam_frames, f)

        test_lps = crop_license_plates(img, lp_detect_model)
        median, bilateral, canny = read_license_plates(test_lps,f)
        ocr_median = ocr(median)
        ocr_bilateral = ocr(bilateral)
        ocr_canny = ocr(canny)
        res.append([f[:-4], ocr_median, ocr_bilateral, ocr_canny])


Processing image_1.jpg


image 1/1 e:\Users\belau\Documents\BV-ML-CV-Praktikum\cam_frames\image_1.jpg: 512x640 1 license_plate, 177.2ms
Speed: 2.0ms preprocess, 177.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 e:\Users\belau\Documents\BV-ML-CV-Praktikum\cam_frames\image_2.jpg: 512x640 1 license_plate, 146.1ms
Speed: 1.0ms preprocess, 146.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


Processing image_2.jpg



image 1/1 e:\Users\belau\Documents\BV-ML-CV-Praktikum\cam_frames\image_3.jpg: 512x640 1 license_plate, 143.1ms
Speed: 1.0ms preprocess, 143.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


Processing image_3.jpg



image 1/1 e:\Users\belau\Documents\BV-ML-CV-Praktikum\cam_frames\image_4.jpg: 512x640 1 license_plate, 153.1ms
Speed: 1.0ms preprocess, 153.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


Processing image_4.jpg



image 1/1 e:\Users\belau\Documents\BV-ML-CV-Praktikum\cam_frames\image_5.jpg: 512x640 1 license_plate, 146.1ms
Speed: 1.0ms preprocess, 146.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


Processing image_5.jpg


In [130]:
import pandas as pd

res = pd.DataFrame(res)
print(res)
res.to_csv("ocr_results.csv", index=False, header=["Image name", "Canny", "Bilateral + Tresh","Median + Tresh"])

             0             1           2          3
0  image_1.jpg     SHGALF206  ISHGRLF206  SWGRUGZUE
1  image_2.jpg      SHGLF206   ISHGLF206   SE8LEZOA
2  image_3.jpg  not detected    SHGLF206   SSCLEATS
3  image_4.jpg     SHGRLF206    SHGLF206  SHESUR20E
4  image_5.jpg     SHGTLF206    SHGLF206  SHESUE20A
